# Python Kernel 

Name: spri (python 3.8)

In [ ]:
print('hello, world')

# ENV Setup and Library Setup

In [ ]:
import yfinance as yf
import torch
import sklearn
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# YFinance Use Case

In [ ]:
stock = yf.Ticker("TSLA")

info = stock.info # has all the additional info about the company but not required for the model

hist = stock.history(period = '60d', interval = '30m') #has the following cols: Data, Open, High, Low, Close, Vol, Dividends and 
#stock splits (dividends and stock splits aren't really required for the model)

#print(hist) #To print the dataset. 

#Additional infomation 
#print(stock.options) #this shows the list of options' exp dates
opt = stock.option_chain(stock.options[0]) #use like this or stock.option_chain("YYYY-MM-DD") Gives details on the Options present. 
#print(opt.calls)
#print(opt.puts)

inc_stmt = stock.income_stmt #Has the income statement with the total revenue of the company
#print(inc_stmt)

rec = stock.recommendations.to_dict("records") #has the recommendations on what to do with the stock 
#print(rec)


news = stock.news #Has all of the news from the yahoo finance page 
#print(news)

In [ ]:
stock.info
key_to_display = ['city', 'state', 'country', 'industry', 'website', 'phone', 'companyOfficers', 'open' , 'previousClose', 'Percentage diff', #for this % diff, just find the how percentage is the opena nd prev close diff
                  'marketCap', 'currency', 'longName', 'timeZoneFullName', 'recommendationKey'
                  ]

In [ ]:
info['companyOfficers'][0]['title']

In [ ]:
new_price = info.get('currentPrice')
old_price = stock.info['previousClose']
percentage_change = ((new_price - old_price) / old_price) * 100
print(str(percentage_change))


In [ ]:
new_price

In [ ]:
stock.info['recommendationKey']

In [ ]:
stock.major_holders.to_dict()['Value']

In [ ]:
stuff_to_display = ['title', 'publisher', 'link', 'relatedTickers', 'thumbnail'] #thumbnail is a dict and you gon get the url like this. stock.news[1]['thumbnail']['resolutions'][0]['url']

stock.news[1]['thumbnail']['resolutions'][0]['url']

In [ ]:
stock.news

In [ ]:
down = stock.upgrades_downgrades
down.reset_index(inplace=True)

In [ ]:
down

In [ ]:
stock.upgrades_downgrades.iloc[:3].reset_index().loc[:, ['Firm', 'ToGrade', 'GradeDate']].to_dict('records')

In [145]:
stock.quarterly_financials.loc[['Total Revenue','Gross Profit', 'Net Income' ,'Cost Of Revenue', 'Operating Revenue', 'Total Expenses']].to_dict()

[{Timestamp('2024-03-31 00:00:00'): 21301000000.0,
  Timestamp('2023-12-31 00:00:00'): 25167000000.0,
  Timestamp('2023-09-30 00:00:00'): 23350000000.0,
  Timestamp('2023-06-30 00:00:00'): 24927000000.0,
  Timestamp('2023-03-31 00:00:00'): 23329000000.0},
 {Timestamp('2024-03-31 00:00:00'): 3696000000.0,
  Timestamp('2023-12-31 00:00:00'): 4438000000.0,
  Timestamp('2023-09-30 00:00:00'): 4178000000.0,
  Timestamp('2023-06-30 00:00:00'): 4533000000.0,
  Timestamp('2023-03-31 00:00:00'): 4511000000.0},
 {Timestamp('2024-03-31 00:00:00'): 1129000000.0,
  Timestamp('2023-12-31 00:00:00'): 7930000000.0,
  Timestamp('2023-09-30 00:00:00'): 1853000000.0,
  Timestamp('2023-06-30 00:00:00'): 2703000000.0,
  Timestamp('2023-03-31 00:00:00'): 2513000000.0},
 {Timestamp('2024-03-31 00:00:00'): 17605000000.0,
  Timestamp('2023-12-31 00:00:00'): 20729000000.0,
  Timestamp('2023-09-30 00:00:00'): 19172000000.0,
  Timestamp('2023-06-30 00:00:00'): 20394000000.0,
  Timestamp('2023-03-31 00:00:00'): 18

In [ ]:
['Total Revenue','Gross Profit', 'Net Income' ,'Cost Of Revenue', 'Operating Revenue', 'Total Expenses']

In [ ]:
rec.to_dict("records")

LINKS:

https://pypi.org/project/yfinance/

https://finance.yahoo.com

# Data Explorations and Idea of Attack 

Now, we got all of these details, how are they going to be used in our model, and what is our model? 

Data:
Open: Opening stock price of the unit time 
Close: Closing stock price of the unit time 
High: Highest stock price of the unit time
Low: Lowest stock price of the unit time 
Volume: Amount of stocks sold in that unit time.

Plan of Attack :

Plan 1: Time series graph: Basically take the Open and Close values and then find the mid value and plot a time series graph and then involue LSTMS for time series grpahs. Good for long forcasts for over a week in the future or so. (Link 2)

Plan 2: Timely Update Graph: Take the open, high and low as X and Close as Y and perform normal LSTM operations. (Link 3)

If you are predicting intraday trading and your interval is 1 min, you need to automatically cache and update the timer and predict the values for the minute. So, you might have to turn your model to Train mode and then feed in this additional info and then turn it back to test mode and predict the outcome. 

In [ ]:
hist

## Setting up Data

In [ ]:
hist_normalized = (hist - hist.min()) / (hist.max() - hist.min())

Y = hist_normalized['Close']
X = hist_normalized.drop('Close', axis=1)
X = X.drop('Dividends', axis=1)
X = X.drop('Stock Splits', axis=1)

X

In [ ]:
X.shape[0]

In [ ]:
X_Values = X.values

#1 is the time step, if you need the lstm to look back the past 1 row and then predict the closing price. then the time step should be 1. 
#if you want the lstm to look back around the past 5 records and then the predict the closing price, then turn the 1 into a 5
X_reshape = X_Values.reshape((X_Values.shape[0], 1, X_Values.shape[1]))
print(X_reshape.shape)
X_reshape

Additional Links

1. https://colab.research.google.com/drive/18WiSw1K0BW3jOKO56vxn11Fo9IyOuRjh#scrollTo=2SoQJk5BYOas

2. https://www.datacamp.com/tutorial/lstm-python-stock-market 

3. https://medium.com/@prajjwalchauhan94017/stock-prediction-and-forecasting-using-lstm-long-short-term-memory-9ff56625de73

# Model

## Model 1 (For plan 2)

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_reshape, Y, test_size=0.0, random_state=42)
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Define the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)
# Evaluate the model
loss = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', loss)


In [ ]:
Inference = np.array([[[0.09978002, 0.07600253, 0.10062503, 0.        ]]])
pred = model(Inference)
yhat_unnormalized = pred.numpy().tolist()[0][0] * (hist['Close'].max() - hist['Close'].min()) + hist['Close'].min()
ytrue_unnmormalized = Y.values[-1] * (hist['Close'].max() - hist['Close'].min()) + hist['Close'].min()
print(yhat_unnormalized, ytrue_unnmormalized)


Idea: The data isn't predicting the future but the time that is already passed. 

* What you need to do is, instead of giving the y as the close price of that row, give the y as the close price of the next row. aka the close price of the next time step. basically you are removing the top value of the close and shifting the total col up by 1 row (this means the first x is pared with the 2nd y and 2nd x is pared by 3rd y and like that the last x won't have an y value) and removing the last x and the using that as the inference x. This might not need testing dataset. 


* or keep everything the same and then take the current, open, high and low price by web scrapping and then predict using that. 

* what you need to is, take 1m as the interval and turn the time step into 30 (so each input is around 30 mintues' worth of data and then you predict the close of the once every 30 mins) (Didn't do exactly this but did 30 min interval and then turn that into 1.5 hour time step.)



* Or just use the open price as x and then web scrap the current price and name it as current open price and then get the y value


Not sure if all of these will work, but try all of them 

## Idea 1 (SORTA WORKING)

In [ ]:
stock = yf.Ticker("GC=F")

info = stock.info # has all the additional info about the company but not required for the model

hist = stock.history(period = '60d', interval = '30m') #has the following cols: Data, Open, High, Low, Close, Vol, Dividends and 
#stock splits (dividends and stock splits aren't really required for the model)

#print(hist) #To print the dataset. 

#Additional infomation 
#print(stock.options) #this shows the list of options' exp dates
#opt = stock.option_chain(stock.options[0]) #use like this or stock.option_chain("YYYY-MM-DD") Gives details on the Options present. 
#print(opt.calls)
#print(opt.puts)

#inc_stmt = stock.income_stmt #Has the income statement with the total revenue of the company
#print(inc_stmt)

#rec = stock.recommendations #has the recommendations on what to do with the stock 
#print(rec)

#BS = stock.balancesheet #Has the balance sheet. 
#print(BS)

#news = stock.news #Has all of the news from the yahoo finance page 
#print(news)

In [ ]:
hist_normalized .index = pd.to_datetime(hist_normalized .index)

# Resample the data to hourly intervals
hourly_data = hist_normalized .resample('H').agg({'Open': 'first',
                                      'High': 'max',
                                      'Low': 'min',
                                      'Close': 'last',
                                      'Volume': 'sum'})

# The 'Dividends' and 'Stock Splits' columns are not applicable for hourly data
# and can be dropped or handled as needed

hourly_data

In [ ]:
hist_normalized = (hist - hist.min()) / (hist.max() - hist.min())

Y = hist_normalized['Close'].iloc[1:]
X = hist_normalized.drop('Close', axis=1)
X = X.drop('Dividends', axis=1)
X = X.drop('Stock Splits', axis=1)
Inference = X.iloc[-1]
X = X.iloc[:-1]

X

In [ ]:
Y

In [ ]:
Inference.values.tolist()

In [ ]:
X_Values = X.values
In = Inference.values
In = In.reshape((1, 1, In.shape[0]))

#1 is the time step, if you need the lstm to look back the past 1 row and then predict the closing price. then the time step should be 1. 
#if you want the lstm to look back around the past 5 records and then the predict the closing price, then turn the 1 into a 5
X_reshape = X_Values.reshape((X_Values.shape[0], 1, X_Values.shape[1]))
print(X_reshape.shape)
X_reshape

In [ ]:
X

In [ ]:
from sklearn.model_selection import train_test_split

# # Split the data
# # X_train, X_test, y_train, y_test = train_test_split(X_reshape, Y, test_size=0.0, random_state=42)
# X_train, y_train = X_reshape, Y
# from keras.models import Sequential
# from keras.layers import LSTM, Dense

# # Define the LSTM model
# model = Sequential()
# model.add(LSTM(50, activation='tanh', input_shape=(X_train.shape[1], X_train.shape[2])))
# model.add(Dense(1))

# # Compile the model
# model.compile(optimizer='adam', loss='mse')

# # Train the model
# model.fit(X_train, y_train, epochs=100)
# # Evaluate the model
# #loss = model.evaluate(X_test, y_test, verbose=0)
# #print('Test loss:', loss)

## Test 2 
from tensorflow.keras.utils import get_custom_objects

from keras import backend as K
from keras.layers import Activation

def custom_activation(x, beta = 1):
        return (K.sigmoid(beta * x) * x)

get_custom_objects().update({'custom_activation': Activation(custom_activation)})

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_reshape, Y, test_size=0.15, random_state=42)
#X_train, y_train = X_reshape, tY
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Define the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Activation(custom_activation,name = "Swish"))
model.add(Dense(1))
#print(model.summary())

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=100)
# Evaluate the model
loss = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', loss)



In [ ]:
pred = model(np.array([[Inference.values.tolist()]]))
yhat_unnormalized = pred.numpy().tolist()[0][0] * (hist['Close'].max() - hist['Close'].min()) + hist['Close'].min()
#ytrue_unnmormalized = 0.02574494974900854 * (hist['Close'].max() - hist['Close'].min()) + hist['Close'].min()
print(yhat_unnormalized) #this value is the predicted value that is in the next timestep future.
#that is the price of the stock in around 2024-04-19 16:00:00-40:00. the predicted price was 398.227. 
# the actual price at that time was 399.12
# the actual stock is actually decreasing.

#Now doing for an hour. the the predicted was 398.44
# the actual price was. 398.38. which is not bad at all!!!!!!!!! 


#Test number two. 
# one hour. the predicted was 399.94. the true was 399.119 and the price is increasing
# 30 mins. The predicted was 399.03 and the true was 399.109. Both were done for msft stock

In [ ]:
hist['Close']

## Idea 2 (Don't think will work.)

## Idea 3 (Works)

In [ ]:
stock = yf.Ticker("msft")

info = stock.info # has all the additional info about the company but not required for the model

hist = stock.history(period = '1mo', interval = '30m') #has the following cols: Data, Open, High, Low, Close, Vol, Dividends and 
#stock splits (dividends and stock splits aren't really required for the model)

#print(hist) #To print the dataset. 

#Additional infomation 
#print(stock.options) #this shows the list of options' exp dates
#opt = stock.option_chain(stock.options[0]) #use like this or stock.option_chain("YYYY-MM-DD") Gives details on the Options present. 
#print(opt.calls)
#print(opt.puts)

#inc_stmt = stock.income_stmt #Has the income statement with the total revenue of the company
#print(inc_stmt)

rec = stock.recommendations #has the recommendations on what to do with the stock 
print(rec)

#BS = stock.balancesheet #Has the balance sheet. 
#print(BS)

news = stock.news #Has all of the news from the yahoo finance page 
print(news)

In [ ]:
hist_normalized = (hist - hist.min()) / (hist.max() - hist.min())

Y = hist_normalized['Close'].iloc[1:]
X = hist_normalized.drop('Close', axis=1)
X = X.drop('Dividends', axis=1)
X = X.drop('Stock Splits', axis=1)
Inference = X.iloc[-1]
X = X.iloc[:-1]

X

In [ ]:
import numpy as np

# Assuming X_Values is your original data
X_Values = X.values

# Initialize an empty list to hold the reshaped data
X_reshape = []

TimeStep = 0 #0 means only the current row, 1 means 1 previous row included. 2 means 2 previous rows included. 
# Loop through each row in the dataset
for i in range(TimeStep, X_Values.shape[0]):
    # Create a window of data that includes the current row and the two preceding rows
    window = X_Values[i-TimeStep:i+1]
    # Reshape the window to fit the LSTM model's input shape
    window_reshaped = window.reshape((1, window.shape[0], window.shape[1]))
    # Append the reshaped window to the list
    X_reshape.append(window_reshaped)

# Convert the list of reshaped windows to a NumPy array
X_reshape = np.concatenate(X_reshape, axis=0)

print(X_reshape.shape)

Y = Y.iloc[TimeStep:]

print(Y.shape)

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data
# X_train, X_test, y_train, y_test = train_test_split(X_reshape, Y, test_size=0.0, random_state=42)
X_train, y_train = X_reshape, Y
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Define the LSTM model
model = Sequential()
model.add(LSTM(50, activation='tanh', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=10)
# Evaluate the model
#loss = model.evaluate(X_test, y_test, verbose=0)
#print('Test loss:', loss)

## Test 2 
# from tensorflow.keras.utils import get_custom_objects

# from keras import backend as K
# from keras.layers import Activation

# def custom_activation(x, beta = 1):
#         return (K.sigmoid(beta * x) * x)

# get_custom_objects().update({'custom_activation': Activation(custom_activation)})

# # Split the data
# X_train, X_test, y_train, y_test = train_test_split(X_reshape, Y, test_size=0.15, random_state=42)
# #X_train, y_train = X_reshape, tY
# from keras.models import Sequential
# from keras.layers import LSTM, Dense

# # Define the LSTM model
# model = Sequential()
# model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
# model.add(Activation(custom_activation,name = "Swish"))
# model.add(Dense(1))
# #print(model.summary())

# # Compile the model
# model.compile(optimizer='adam', loss='mse')

# # Train the model
# model.fit(X_train, y_train, epochs=100)
# # Evaluate the model
# loss = model.evaluate(X_test, y_test, verbose=0)
# print('Test loss:', loss)



In [ ]:
In = X_reshape[-1]
In = np.delete(In, 0, axis=0)
In = np.append(In, np.array([Inference.values.tolist()]), axis=0)
In = In[np.newaxis, ...]

In [ ]:
pred = model(In)
yhat_unnormalized = pred.numpy().tolist()[0][0] * (hist['Close'].max() - hist['Close'].min()) + hist['Close'].min()
#ytrue_unnmormalized = 0.02574494974900854 * (hist['Close'].max() - hist['Close'].min()) + hist['Close'].min()
print(yhat_unnormalized)

## Idea 4 (Should try)

## Idea 5 (1 Day model)

In [146]:
stock = yf.Ticker("MSFT")

info = stock.info # has all the additional info about the company but not required for the model

hist = stock.history(period = '1y', interval = '1d') #has the following cols: Data, Open, High, Low, Close, Vol, Dividends and 
#stock splits (dividends and stock splits aren't really required for the model)

#print(hist) #To print the dataset. 

#Additional infomation 
#print(stock.options) #this shows the list of options' exp dates
opt = stock.option_chain(stock.options[0]) #use like this or stock.option_chain("YYYY-MM-DD") Gives details on the Options present. 
#print(opt.calls)
#print(opt.puts)

inc_stmt = stock.income_stmt #Has the income statement with the total revenue of the company
#print(inc_stmt)

rec = stock.recommendations #has the recommendations on what to do with the stock 
#print(rec)

BS = stock.balancesheet #Has the balance sheet. 
#print(BS)

news = stock.news #Has all of the news from the yahoo finance page 
#print(news)

In [ ]:
hist

In [156]:
hist_normalized = (hist - hist.min()) / (hist.max() - hist.min())
hist_inference = hist_normalized.iloc[-1]
hist_normalized = hist_normalized.iloc[:-1]

Y = hist_normalized['Close']
X = hist_normalized.drop('Close', axis=1)
X = X.drop('Dividends', axis=1)
X = X.drop('Stock Splits', axis=1)


X_Values = X.values

#1 is the time step, if you need the lstm to look back the past 1 row and then predict the closing price. then the time step should be 1. 
#if you want the lstm to look back around the past 5 records and then the predict the closing price, then turn the 1 into a 5
X_reshape = X_Values.reshape((X_Values.shape[0], 1, X_Values.shape[1]))
print(X_reshape.shape)
X_reshape

ValueError: No axis named 1 for object type Series

In [148]:
from sklearn.model_selection import train_test_split

# Split the data
# X_train, X_test, y_train, y_test = train_test_split(X_reshape, Y, test_size=0.0, random_state=42)
X_train, y_train = X_reshape, Y
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Define the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)
# Evaluate the model
#loss = model.evaluate(X_test, y_test, verbose=0)
#print('Test loss:', loss)


Epoch 1/100
8/8 [==============================] - 7s 9ms/step - loss: 0.2805
Epoch 2/100
8/8 [==============================] - 0s 8ms/step - loss: 0.2420
Epoch 3/100
8/8 [==============================] - 0s 6ms/step - loss: 0.2075
Epoch 4/100
8/8 [==============================] - 0s 8ms/step - loss: 0.1726
Epoch 5/100
8/8 [==============================] - 0s 5ms/step - loss: 0.1405
Epoch 6/100
8/8 [==============================] - 0s 5ms/step - loss: 0.1104
Epoch 7/100
8/8 [==============================] - 0s 5ms/step - loss: 0.0802
Epoch 8/100
8/8 [==============================] - 0s 5ms/step - loss: 0.0544
Epoch 9/100
8/8 [==============================] - 0s 6ms/step - loss: 0.0344
Epoch 10/100
8/8 [==============================] - 0s 5ms/step - loss: 0.0223
Epoch 11/100
8/8 [==============================] - 0s 5ms/step - loss: 0.0157
Epoch 12/100
8/8 [==============================] - 0s 5ms/step - loss: 0.0133
Epoch 13/100
8/8 [==============================] - 0s 5ms/st

In [160]:
hist_inference.drop('Close')[:4]

Open      0.776143
High      0.771675
Low       0.743642
Volume    0.020203
Name: 2024-04-30 00:00:00-04:00, dtype: float64

In [161]:

pred = model(hist_inference.drop('Close')[:4].values.reshape((1, 1, hist_inference.drop('Close')[:4].shape[0])))
yhat_unnormalized = pred.numpy().tolist()[0][0] * (hist['Close'].max() - hist['Close'].min()) + hist['Close'].min()
#ytrue_unnmormalized = Y.values[-1] * (hist['Close'].max() - hist['Close'].min()) + hist['Close'].min()
print(yhat_unnormalized)
#now the predicted price is for the end of the day for 20th 4 2024 for msft. and the predicted price is 403.98
# and actual end price was 399.12

397.3362390546972


# Model 2

In [ ]:
stock = yf.Ticker("MSFT")

info = stock.info # has all the additional info about the company but not required for the model

hist = stock.history(period = 'max', interval = '1d') #has the following cols: Data, Open, High, Low, Close, Vol, Dividends and 
#stock splits (dividends and stock splits aren't really required for the model)

#print(hist) #To print the dataset. 

#Additional infomation 
#print(stock.options) #this shows the list of options' exp dates
opt = stock.option_chain(stock.options[0]) #use like this or stock.option_chain("YYYY-MM-DD") Gives details on the Options present. 
#print(opt.calls)
#print(opt.puts)

inc_stmt = stock.income_stmt #Has the income statement with the total revenue of the company
#print(inc_stmt)

rec = stock.recommendations #has the recommendations on what to do with the stock 
#print(rec)

BS = stock.balancesheet #Has the balance sheet. 
#print(BS)

news = stock.news #Has all of the news from the yahoo finance page 
#print(news)

In [ ]:
hist

In [ ]:
plt.plot(range(hist.shape[0]), (hist['High'] + hist['Low'])/2.0)
plt.show()

In [ ]:
High = hist.loc[:,'High']
Low = hist.loc[:,'Low']
Mid = (High+Low)/2

In [ ]:
len(Mid)

In [ ]:
train = Mid[:9500]
test = Mid[9500:]

# To Do

1. Clean up and ready the dataset for both Plan 1 and 2. 